In [1]:
import os, sys, time, pickle, json
import argparse
from tqdm import tqdm
from collections import defaultdict
import numpy as np
import re
import torch
from datasets import load_from_disk
from transformers import ViTForImageClassification
from utils.helper import get_device, json2dict
from utils.vit_util import transforms, transforms_c100, ViTFromLastLayer, identfy_tgt_misclf, get_ori_model_predictions, get_new_model_predictions, get_batched_hs, get_batched_labels, sample_from_correct_samples, sample_true_positive_indices_per_class, maybe_initialize_repair_weights_
from utils.constant import ViTExperiment, Experiment3
from utils.log import set_exp_logging

2025-07-31 04:09:57.515344: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-31 04:09:57.598621: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-31 04:09:58.122734: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2025-07-31 04:09:58.122797: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l

In [2]:
TGT_SPLIT = "repair"
TGT_LAYER = 11
DEFAULT_SETTINGS = {
    "n": 5, 
    "ratio_sampled_from_correct": 0.05,  # 正解サンプルのうち何割をrepairに使うか
    "num_sampled_from_correct": 200,  # 正解サンプルのうち何サンプルをrepairに使うか
    "max_search_num": 50,
    "pop_size": 100,
    "alpha": 0.5
}

In [3]:
def main(argv=None):
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument("ds", type=str)
    parser.add_argument('k', type=int)
    parser.add_argument('tgt_rank', type=int)
    parser.add_argument("wnum", type=int)
    parser.add_argument("--setting_path", type=str, default=None)
    parser.add_argument("--fl_method", type=str, default="vdiff")
    parser.add_argument('--misclf_type', type=str, default="tgt")
    parser.add_argument('--only_eval', action='store_true', default=False)
    parser.add_argument("--custom_alpha", type=float, default=None)
    parser.add_argument("--include_other_TP_for_fitness", action='store_true', default=False)
    parser.add_argument("--fpfn", type=str, default=None, choices=["fp", "fn"])
    parser.add_argument("--custom_bounds", type=str, default=None, choices=["Arachne", "ContrRep"])
    args = parser.parse_args(argv)
    # 引数の取得
    ds_name = args.ds
    k = args.k
    tgt_rank = args.tgt_rank
    wnum = args.wnum
    setting_path = args.setting_path
    fl_method = args.fl_method
    misclf_type = args.misclf_type
    only_eval = args.only_eval
    custom_alpha = args.custom_alpha
    include_other_TP_for_fitness = args.include_other_TP_for_fitness
    fpfn = args.fpfn
    custom_bounds = args.custom_bounds
    # print(f"ds_name: {ds_name}, k: {k}, tgt_rank: {tgt_rank}, setting_path: {setting_path}, fl_method: {fl_method}, misclf_type: {misclf_type}, only_eval: {only_eval}, custom_alpha: {custom_alpha}, include_other_TP_for_fitness: {include_other_TP_for_fitness}, fpfn: {fpfn}, custom_bounds: {custom_bounds}")

    # 設定のjsonファイルが指定された場合
    if setting_path is not None:
        assert os.path.exists(setting_path), f"{setting_path} does not exist."
        setting_dic = json2dict(setting_path)
        # setting_idは setting_{setting_id}.json というファイル名になる
        setting_id = os.path.basename(setting_path).split(".")[0].split("_")[-1]
    # 設定のjsonファイルが指定されない場合はnとalphaだけカスタムorデフォルトの設定を使う
    else:
        setting_dic = DEFAULT_SETTINGS
        # custom_alpha, custom_boundsが1つでも指定されている場合はいったん空文字にする
        setting_id = "default" if (custom_alpha is None) and (custom_bounds is None) else ""
        parts = []
        if wnum is not None:
            setting_dic["wnum"] = wnum
            parts.append(f"n{wnum}")
        if custom_alpha is not None:
            setting_dic["alpha"] = custom_alpha
            parts.append(f"alpha{custom_alpha}")
        if custom_bounds is not None:
            setting_dic["bounds"] = custom_bounds
            parts.append(f"bounds{custom_bounds}")
        # リストの要素を'_'で連結
        setting_id = "_".join(parts)
    # pretrained modelのディレクトリ
    pretrained_dir = getattr(ViTExperiment, ds_name.replace("-", "_")).OUTPUT_DIR.format(k=k)
    # 結果とかログの保存先を先に作っておく
    # save_dirは, 5種類の誤分類タイプのどれかを一意に表す
    if fpfn is not None and misclf_type == "tgt": # tgt_fp or tgt_fn
        save_dir = os.path.join(pretrained_dir, f"misclf_top{tgt_rank}", f"{misclf_type}_{fpfn}_repair_weight_by_de")
    elif misclf_type == "all": # all
        save_dir = os.path.join(pretrained_dir, f"{misclf_type}_repair_weight_by_de")
    else: # tgt_all or src_tgt
        save_dir = os.path.join(pretrained_dir, f"misclf_top{tgt_rank}", f"{misclf_type}_repair_weight_by_de")
    os.makedirs(save_dir, exist_ok=True)
    
    
    # print(f"ds_name: {ds_name}, fold_id: {k}, setting_path: {setting_path}")
    # print(f"setting_dic (id={setting_id}): {setting_dic}")

    # datasetごとに違う変数のセット
    if ds_name == "c10" or ds_name == "tiny-imagenet":
        tf_func = transforms
        label_col = "label"
    elif ds_name == "c100":
        tf_func = transforms_c100
        label_col = "fine_label"
    else:
        NotImplementedError
    tgt_pos = ViTExperiment.CLS_IDX
    ds_dirname = f"{ds_name}_fold{k}"
    # デバイス (cuda, or cpu) の取得
    device = get_device()
    # datasetをロード (初回の読み込みだけやや時間かかる)
    ds = load_from_disk(os.path.join(ViTExperiment.DATASET_DIR, ds_dirname))
    # ラベルの取得 (shuffleされない)
    labels = {
        "train": np.array(ds["train"][label_col]),
        "repair": np.array(ds["repair"][label_col]),
        "test": np.array(ds["test"][label_col])
    }
    # 読み込まれた時にリアルタイムで前処理を適用するようにする
    ds_preprocessed = ds.with_transform(tf_func)
    # pretrained modelのロード
    model, loading_info = ViTForImageClassification.from_pretrained(pretrained_dir, output_loading_info=True)
    model.to(device).eval()
    model = maybe_initialize_repair_weights_(model, loading_info["missing_keys"])
    end_li = model.vit.config.num_hidden_layers
    batch_size = ViTExperiment.BATCH_SIZE
    tgt_split = TGT_SPLIT # NOTE: we only use repair split for repairing
    ori_tgt_ds = ds_preprocessed[tgt_split]
    ori_tgt_labels = labels[tgt_split]
    tgt_layer = TGT_LAYER # NOTE: we only use the last layer for repairing
    print(f"tgt_layer: {tgt_layer}, tgt_split: {tgt_split}")

    # ===============================================
    # localization phase
    # ===============================================

    if fpfn is not None and misclf_type == "tgt":
        location_save_dir = os.path.join(pretrained_dir, f"misclf_top{tgt_rank}", f"{misclf_type}_{fpfn}_weights_location")
    elif misclf_type == "all":
        location_save_dir = os.path.join(pretrained_dir, f"all_weights_location")
    else:
        location_save_dir = os.path.join(pretrained_dir, f"misclf_top{tgt_rank}", f"{misclf_type}_weights_location")
    # 重みの位置情報の保存ファイル
    if fl_method == "ours" or fl_method == "bl":
        if wnum == 11:
            if fl_method == "ours":
                location_filename = f"exp-repair-3-2_location_n{wnum}_weight_ours.npy"
            elif fl_method == "bl":
                location_filename = "exp-fl-2_location_pareto_weight_bl.npy"
        else:
            location_filename = f"exp-repair-4-1_location_n{wnum}_weight_{fl_method}.npy"
        location_save_path = os.path.join(location_save_dir, location_filename)
    elif fl_method == "random":
        raise NotImplementedError("Random method is not implemented for localization phase.")
    assert os.path.exists(location_save_path), f"{location_save_path} does not exist. Please run the localization phase first."
    pos_before, pos_after = np.load(location_save_path, allow_pickle=True)
    # log表示
    # print(f"pos_before={pos_before}")
    # print(f"pos_after={pos_after}")
    # print(f"num(pos_to_fix)=num(pos_before)+num(pos_before)={len(pos_before)}+{len(pos_after)}={len(pos_before)+len(pos_after)}")
    # print(f"num(pos_to_fix)=num(pos_before)+num(pos_before)={len(pos_before)}+{len(pos_after)}={len(pos_before)+len(pos_after)}")
    return pos_before, pos_after

In [4]:
from itertools import product

# 結果格納用リスト（ヒートマップ用）
results = {}

ds_list = ["c100", "tiny-imagenet"]
k_list = [0]
tgt_rank_list = [1, 2, 3]
misclf_type_list = ["src_tgt", "tgt"]
fpfn_list = [None, "fp", "fn"]
w_num_list = [11, 236, 472, 944]
fl_method_list = ["ours", "bl"]  # "random"は未対応なので外す
alpha = float(10 / 11)

for ds, k, tgt_rank, misclf_type, fpfn, w_num, fl_method in product(ds_list, k_list, tgt_rank_list, misclf_type_list, fpfn_list, w_num_list, fl_method_list):
    if (misclf_type == "src_tgt" or misclf_type == "all") and fpfn is not None:
        continue
    if misclf_type == "tgt" and fpfn is None:
        continue

    print(f"{'='*90}\nProcessing: ds={ds}, k={k}, tgt_rank={tgt_rank}, alpha={alpha}, misclf_type={misclf_type}, fpfn={fpfn}, w_num={w_num}, fl_method={fl_method}")

    # 引数をリスト形式で作成
    argv = [
        ds,
        str(k),
        str(tgt_rank),
        str(w_num),
        "--fl_method", fl_method,
        "--misclf_type", misclf_type,
        "--custom_alpha", str(alpha)
    ]
    if fpfn is not None:
        argv.extend(["--fpfn", fpfn])

    # 実行
    try:
        pos_before, pos_after = main(argv)
        results[(ds, k, tgt_rank, misclf_type, fpfn, w_num, fl_method)] = (pos_before, pos_after)
        # print(results)
    except Exception as e:
        print(f"Error occurred for config: {argv}")
        print(e)

Processing: ds=c100, k=0, tgt_rank=1, alpha=0.9090909090909091, misclf_type=src_tgt, fpfn=None, w_num=11, fl_method=ours
Device: cuda


Some weights of ViTForImageClassification were not initialized from the model checkpoint at /src/src/out_vit_c100_fold0 and are newly initialized: ['vit.encoder.layer.11.intermediate.repair.weight', 'vit.encoder.layer.3.intermediate.repair.weight', 'vit.encoder.layer.5.intermediate.repair.weight', 'vit.encoder.layer.8.intermediate.repair.weight', 'vit.encoder.layer.7.intermediate.repair.weight', 'vit.encoder.layer.6.intermediate.repair.weight', 'vit.encoder.layer.2.intermediate.repair.weight', 'vit.encoder.layer.9.intermediate.repair.weight', 'vit.encoder.layer.10.intermediate.repair.weight', 'vit.encoder.layer.0.intermediate.repair.weight', 'vit.encoder.layer.4.intermediate.repair.weight', 'vit.encoder.layer.1.intermediate.repair.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🛠️ Initializing intermediate.repair.weight as identity matrix (for missing weights)
tgt_layer: 11, tgt_split: repair
Processing: ds=c100, k=0, tgt_rank=1, alpha=0.9090909090909091, misclf_type=src_tgt, fpfn=None, w_num=11, fl_method=bl
Device: cuda


Some weights of ViTForImageClassification were not initialized from the model checkpoint at /src/src/out_vit_c100_fold0 and are newly initialized: ['vit.encoder.layer.11.intermediate.repair.weight', 'vit.encoder.layer.3.intermediate.repair.weight', 'vit.encoder.layer.5.intermediate.repair.weight', 'vit.encoder.layer.8.intermediate.repair.weight', 'vit.encoder.layer.7.intermediate.repair.weight', 'vit.encoder.layer.6.intermediate.repair.weight', 'vit.encoder.layer.2.intermediate.repair.weight', 'vit.encoder.layer.9.intermediate.repair.weight', 'vit.encoder.layer.10.intermediate.repair.weight', 'vit.encoder.layer.0.intermediate.repair.weight', 'vit.encoder.layer.4.intermediate.repair.weight', 'vit.encoder.layer.1.intermediate.repair.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🛠️ Initializing intermediate.repair.weight as identity matrix (for missing weights)
tgt_layer: 11, tgt_split: repair
Processing: ds=c100, k=0, tgt_rank=1, alpha=0.9090909090909091, misclf_type=src_tgt, fpfn=None, w_num=236, fl_method=ours
Device: cuda


...

🛠️ Initializing intermediate.repair.weight as identity matrix (for missing weights)
tgt_layer: 11, tgt_split: repair
Processing: ds=c100, k=0, tgt_rank=3, alpha=0.9090909090909091, misclf_type=tgt, fpfn=fn, w_num=944, fl_method=ours
Device: cuda


Some weights of ViTForImageClassification were not initialized from the model checkpoint at /src/src/out_vit_c100_fold0 and are newly initialized: ['vit.encoder.layer.11.intermediate.repair.weight', 'vit.encoder.layer.3.intermediate.repair.weight', 'vit.encoder.layer.5.intermediate.repair.weight', 'vit.encoder.layer.8.intermediate.repair.weight', 'vit.encoder.layer.7.intermediate.repair.weight', 'vit.encoder.layer.6.intermediate.repair.weight', 'vit.encoder.layer.2.intermediate.repair.weight', 'vit.encoder.layer.9.intermediate.repair.weight', 'vit.encoder.layer.10.intermediate.repair.weight', 'vit.encoder.layer.0.intermediate.repair.weight', 'vit.encoder.layer.4.intermediate.repair.weight', 'vit.encoder.layer.1.intermediate.repair.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🛠️ Initializing intermediate.repair.weight as identity matrix (for missing weights)
tgt_layer: 11, tgt_split: repair
Processing: ds=c100, k=0, tgt_rank=3, alpha=0.9090909090909091, misclf_type=tgt, fpfn=fn, w_num=944, fl_method=bl
Device: cuda


Some weights of ViTForImageClassification were not initialized from the model checkpoint at /src/src/out_vit_c100_fold0 and are newly initialized: ['vit.encoder.layer.11.intermediate.repair.weight', 'vit.encoder.layer.3.intermediate.repair.weight', 'vit.encoder.layer.5.intermediate.repair.weight', 'vit.encoder.layer.8.intermediate.repair.weight', 'vit.encoder.layer.7.intermediate.repair.weight', 'vit.encoder.layer.6.intermediate.repair.weight', 'vit.encoder.layer.2.intermediate.repair.weight', 'vit.encoder.layer.9.intermediate.repair.weight', 'vit.encoder.layer.10.intermediate.repair.weight', 'vit.encoder.layer.0.intermediate.repair.weight', 'vit.encoder.layer.4.intermediate.repair.weight', 'vit.encoder.layer.1.intermediate.repair.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


🛠️ Initializing intermediate.repair.weight as identity matrix (for missing weights)
tgt_layer: 11, tgt_split: repair
Processing: ds=tiny-imagenet, k=0, tgt_rank=1, alpha=0.9090909090909091, misclf_type=src_tgt, fpfn=None, w_num=11, fl_method=ours
Device: cuda
tgt_layer: 11, tgt_split: repair
Processing: ds=tiny-imagenet, k=0, tgt_rank=1, alpha=0.9090909090909091, misclf_type=src_tgt, fpfn=None, w_num=11, fl_method=bl
Device: cuda
tgt_layer: 11, tgt_split: repair
Processing: ds=tiny-imagenet, k=0, tgt_rank=1, alpha=0.9090909090909091, misclf_type=src_tgt, fpfn=None, w_num=236, fl_method=ours
Device: cuda
tgt_layer: 11, tgt_split: repair
Processing: ds=tiny-imagenet, k=0, tgt_rank=1, alpha=0.9090909090909091, misclf_type=src_tgt, fpfn=None, w_num=236, fl_method=bl
Device: cuda
tgt_layer: 11, tgt_split: repair
Processing: ds=tiny-imagenet, k=0, tgt_rank=1, alpha=0.9090909090909091, misclf_type=src_tgt, fpfn=None, w_num=472, fl_method=ours
Device: cuda
tgt_layer: 11, tgt_split: repair
Proc

In [48]:
def plot_jaccard_ours_vs_bl(results, save_path="jaccard_heatmap.pdf"):
    import matplotlib.pyplot as plt
    import seaborn as sns
    import numpy as np
    from collections import defaultdict
    from matplotlib import gridspec

    jaccard_dict = {}

    for key, value in results.items():
        ds, k, tgt_rank, misclf_type, fpfn, w_num, fl_method = key

        ds_label = {"c100": "C100", "tiny-imagenet": "TinyImg"}.get(ds, ds)

        if misclf_type == "tgt" and fpfn:
            misclf_label = f"TGT-{fpfn.upper()}"
        elif misclf_type == "src_tgt":
            misclf_label = "SRC-TGT"
        else:
            misclf_label = misclf_type.upper()

        label = (ds_label, tgt_rank, misclf_label, w_num)

        # pos_before → (0, i, j), pos_after → (1, i, j)
        full_pos = set((0, *p) for p in value[0]) | set((1, *p) for p in value[1])
        if not w_num == 11:
            assert len(full_pos) == w_num, f"len(full_pos): {len(full_pos)}, w_num: {w_num}"

        if fl_method == "ours":
            jaccard_dict.setdefault(label, {})["ours"] = full_pos
        elif fl_method == "bl":
            jaccard_dict.setdefault(label, {})["bl"] = full_pos

    data = []
    labels = []
    for key, d in jaccard_dict.items():
        if "ours" in d and "bl" in d:
            ours, bl = d["ours"], d["bl"]
            intersection = len(ours & bl)
            union = len(ours | bl)
            jacc = intersection / union if union > 0 else 0.0
            data.append(jacc)
            ds_label, r, misclf_label, w = key
            if not w == 11:
                assert len(ours) == len(bl) == w, f"len(bl): {len(bl)}, len(ours): {len(ours)}, w_num: {w}"
            setting_label = f"{ds_label}\nRank {r}\n{misclf_label}"
            labels.append((w, setting_label))

    # ヒートマップデータ構築
    heatmap_dict = defaultdict(dict)
    for (wnum, label), jacc in zip(labels, data):
        heatmap_dict[wnum][label] = jacc

    sorted_ws = sorted(heatmap_dict.keys())
    sorted_labels = sorted(set(k for v in heatmap_dict.values() for k in v))

    matrix = np.zeros((len(sorted_ws), len(sorted_labels)))
    for i, w in enumerate(sorted_ws):
        for j, s in enumerate(sorted_labels):
            matrix[i, j] = heatmap_dict[w].get(s, np.nan)

    fig = plt.figure(figsize=(1.2 * len(sorted_labels), 0.8 * len(sorted_ws) + 1))
    gs = gridspec.GridSpec(1, 2, width_ratios=[20, 1], wspace=0.05)
    ax = fig.add_subplot(gs[0])
    cbar_ax = fig.add_subplot(gs[1])

    sns.heatmap(matrix, annot=True, xticklabels=sorted_labels, yticklabels=sorted_ws,
                cmap="Greys", fmt=".2f", ax=ax, cbar_ax=cbar_ax, annot_kws={"fontsize": 12})

    ax.set_ylabel("Number of Weights (N_w)", fontsize=12)
    ax.set_xlabel("")
    ax.set_title("")
    ax.tick_params(axis='x', labelrotation=0, labelsize=12)
    ax.tick_params(axis='y', labelrotation=0, labelsize=12)

    plt.savefig(save_path, bbox_inches="tight", pad_inches=0.1)
    print(f"Saved heatmap to {save_path}")
    plt.close(fig)

In [50]:
plot_jaccard_ours_vs_bl(results, save_path="discuss_suspw.pdf")


Saved heatmap to discuss_suspw.pdf
